In [37]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [38]:
df = pd.read_csv('selected_features_without_text.csv')

In [39]:
df

,Unnamed: 0,reviewID,reviewerID,restaurantID,flagged,reviewerFriendCount,reviewerNumReviews,reviewerUsefulCount,resReviewCount,resRating,...,adj_count,polarity,subjectivity,main_topic,reviewRating_1,reviewRating_2,reviewRating_3,reviewRating_4,reviewRating_5,reviewRating
0,0,GtwU21YOQn-wf4vWRUIx6w,bNYesZ944s6IJVowOnB0iA,pbEiXam9YJL3neCYHGwLUA,0,0.002337,0.018008,0.001885,841,4.5,...,0.427350,1.391395e-01,0.528655,0.868421,False,False,False,False,True,5
1,1,0LpVTc3,TRKxLC3y-ZvP45e5iilMtw,pbEiXam9YJL3neCYHGwLUA,0,0.000000,0.001149,0.000000,841,4.5,...,0.068376,4.666667e-01,0.566667,0.263158,False,False,False,False,True,5
2,2,tljtLzf68Fkwf,0EMm8umAqXZzyhxNpL4M9g,pbEiXam9YJL3neCYHGwLUA,0,0.000425,0.003831,0.000276,841,4.5,...,0.042735,4.133333e-01,0.720000,0.473684,False,False,True,False,False,3
3,3,iSN,DlwexC7z88ymAzu45skODw,pbEiXam9YJL3neCYHGwLUA,0,0.017850,0.022605,0.001609,841,4.5,...,0.290598,1.205482e-01,0.535201,0.868421,False,False,True,False,False,3
4,4,Jmwrh7,kW2dk1CWihmh3g7k9N2G8A,pbEiXam9YJL3neCYHGwLUA,0,0.070761,0.462835,0.075869,841,4.5,...,0.170940,1.674603e-01,0.564683,0.868421,False,False,False,False,True,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26951,26951,PZu8sDx2T2,tivh8lr6pzBDNfrJLYWh_g,v9qEDxi3t-P0CmGWAkkGvw,0,0.006587,0.062835,0.009242,14,4.0,...,0.051282,2.266234e-01,0.760173,0.763158,False,False,False,True,False,4
26952,26952,S-zbPPGoB,jKs4FQgkV0wSX8BG2_dgTg,RRflazDtBkqqpvEz2hbV2w,0,0.001275,0.003831,0.000322,15,4.0,...,0.076923,2.692708e-01,0.608333,0.894737,False,False,False,False,True,5
26953,26953,roKqXYooTy49OMAIJJjf,vX6aOMQ3HWCbwZVfCkCauw,zI0E_yruu58ea-xq9aHi-w,0,0.002762,0.018391,0.002667,16,4.0,...,0.042735,-1.190476e-01,0.588095,0.263158,False,False,True,False,False,3
26954,26954,FefmFaWa,vX6aOMQ3HWCbwZVfCkCauw,6XVXM78gBuU3gpq2hTOgJA,0,0.002762,0.018391,0.002667,17,4.0,...,0.042735,6.938894e-18,0.658333,0.552632,False,False,True,False,False,3


In [40]:
df.columns

Index(['Unnamed: 0', 'reviewID', 'reviewerID', 'restaurantID', 'flagged',
       'reviewerFriendCount', 'reviewerNumReviews', 'reviewerUsefulCount',
       'resReviewCount', 'resRating', 'date_diff_joinedandreviewed',
       'Num_Punc', 'adj_count', 'polarity', 'subjectivity', 'main_topic',
       'reviewRating_1', 'reviewRating_2', 'reviewRating_3', 'reviewRating_4',
       'reviewRating_5', 'reviewRating'],
      dtype='object')

Train Test Split

In [47]:
from sklearn.model_selection import GroupShuffleSplit

# remove target variable
X = df.drop(['flagged'], axis=1) 

y = df.flagged

splitter = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=42)
split = splitter.split(X, groups=X['reviewerID'])
train_inds, test_inds = next(split)

X_train = X.iloc[train_inds]
X_test = X.iloc[test_inds]
y_train = y.iloc[train_inds]
y_test = y.iloc[test_inds]

# remove non-numerical variables from X sets
X_train_ohe = X_train.drop(['Unnamed: 0', 'reviewID', 'reviewerID', 'restaurantID', 'reviewRating_1', 'reviewRating_2', 'reviewRating_3', 'reviewRating_4', 'reviewRating_5'], axis=1)
X_train_rating = X_train.drop(['Unnamed: 0', 'reviewID', 'reviewerID', 'restaurantID', 'reviewRating'], axis=1)
X_train_all = X_train.drop(['Unnamed: 0', 'reviewID', 'reviewerID', 'restaurantID'], axis=1)

X_test_ohe = X_test.drop(['Unnamed: 0', 'reviewID', 'reviewerID', 'restaurantID', 'reviewRating_1', 'reviewRating_2', 'reviewRating_3', 'reviewRating_4', 'reviewRating_5'], axis=1)
X_test_rating = X_test.drop(['Unnamed: 0', 'reviewID', 'reviewerID', 'restaurantID', 'reviewRating'], axis=1)
X_test_all = X_test.drop(['Unnamed: 0', 'reviewID', 'reviewerID', 'restaurantID'], axis=1)

In [42]:
X_train_rating.columns

Index(['reviewerFriendCount', 'reviewerNumReviews', 'reviewerUsefulCount',
       'resReviewCount', 'resRating', 'date_diff_joinedandreviewed',
       'Num_Punc', 'adj_count', 'polarity', 'subjectivity', 'main_topic',
       'reviewRating_1', 'reviewRating_2', 'reviewRating_3', 'reviewRating_4',
       'reviewRating_5'],
      dtype='object')

Logistic Regression Model (OHE)

In [43]:
model_ohe = LogisticRegression(max_iter=10000)
model_ohe.fit(X_train_ohe, y_train)

y_pred_ohe = model_ohe.predict(X_test_ohe)

Results Analysis

In [44]:
print("Accuracy:", accuracy_score(y_test, y_pred_ohe))
print('-----------------------------------------------------')
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_ohe))
print('-----------------------------------------------------')
print("Classification Report:\n", classification_report(y_test, y_pred_ohe))

Accuracy: 0.8276176577891281
-----------------------------------------------------
Confusion Matrix:
 [[4057  228]
 [ 717  480]]
-----------------------------------------------------
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.95      0.90      4285
           1       0.68      0.40      0.50      1197

    accuracy                           0.83      5482
   macro avg       0.76      0.67      0.70      5482
weighted avg       0.81      0.83      0.81      5482



Logistic Regression Model (Rating)

In [45]:
model_rating = LogisticRegression(max_iter=10000)
model_rating.fit(X_train_rating, y_train)

y_pred_rating = model_rating.predict(X_test_rating)

In [46]:
print("Accuracy:", accuracy_score(y_test, y_pred_rating))
print('-----------------------------------------------------')
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rating))
print('-----------------------------------------------------')
print("Classification Report:\n", classification_report(y_test, y_pred_rating))

Accuracy: 0.8267055819044145
-----------------------------------------------------
Confusion Matrix:
 [[4058  227]
 [ 723  474]]
-----------------------------------------------------
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.95      0.90      4285
           1       0.68      0.40      0.50      1197

    accuracy                           0.83      5482
   macro avg       0.76      0.67      0.70      5482
weighted avg       0.81      0.83      0.81      5482



In [48]:
model_all = LogisticRegression(max_iter=10000)
model_all.fit(X_train_all, y_train)

y_pred_all = model_all.predict(X_test_all)

print("Accuracy:", accuracy_score(y_test, y_pred_all))
print('-----------------------------------------------------')
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_all))
print('-----------------------------------------------------')
print("Classification Report:\n", classification_report(y_test, y_pred_all))

Accuracy: 0.8287121488507844
-----------------------------------------------------
Confusion Matrix:
 [[4064  221]
 [ 718  479]]
-----------------------------------------------------
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.95      0.90      4285
           1       0.68      0.40      0.51      1197

    accuracy                           0.83      5482
   macro avg       0.77      0.67      0.70      5482
weighted avg       0.81      0.83      0.81      5482

